# Code for gathering the NAATBatt applicable facilities and mapping census demographic data with them

_____________________________________________________________________________________________________________

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
from plotly import graph_objects as go
import geopandas as gpd
import fiona
from census import Census
import us
from datetime import datetime
import censusdata
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="http")
import string
import re
from pandas import ExcelWriter

## Import NAATBatt data and get location information needed to map with census data

In [ ]:
# state and county fip data
df_fips = pd.read_csv("Data/US_FIPS_Codes_State_County.csv",skiprows=1,converters={'FIPS State': str,'FIPS County': str})
state_fips_codes = {
    'WA': '53', 'DE': '10', 'DC': '11', 'WI': '55', 'WV': '54', 'HI': '15',
    'FL': '12', 'WY': '56', 'NJ': '34', 'NM': '35', 'TX': '48', 'SD': '46',
    'LA': '22', 'NC': '37', 'ND': '38', 'NE': '31', 'TN': '47', 'NY': '36',
    'PA': '42', 'AK': '02', 'NV': '32', 'NH': '33', 'VA': '51', 'CO': '08',
    'CA': '06', 'AL': '01', 'AR': '05', 'VT': '50', 'IL': '17', 'GA': '13',
    'IN': '18', 'IA': '19', 'MA': '25', 'AZ': '04', 'ID': '16', 'CT': '09',
    'ME': '23', 'MD': '24', 'OK': '40', 'OH': '39', 'UT': '49', 'MO': '29',
    'MN': '27', 'MI': '26', 'RI': '44', 'KS': '20', 'MT': '30', 'MS': '28',
    'SC': '45', 'KY': '21', 'OR': '41'}


# sheet names to loop through NAATBatt data set
sheet_names = ['1-RawMatl','2-BGMatl','3-BComp','4-ElectrodeCell','5-ModPack','6-EOL']
# creating to add updated NAATBatt dfs with state and county fips data
dfs = []
# creating to get a list of all state and county fips which will be used to pull census data at the block group level!
state_county_FIP = []

# loop through all the sheets in NAATBatt
for x in sheet_names:
    # read in NAATBatt data
    df = pd.read_excel('Data/naatbatt_clean_locations/naatbatt-for-census-gathering.xlsx',sheet_name=x)
    # create new columns to save FIP data
    df['County'] = ''
    df['State_FIPS'] = ''
    df['County_FIPS'] = ''
    df['State_County_FIPS'] = ''

    # loop through all rows in the df (individual sheet from NAATBatt)
    for i,r in df.iterrows():
        # get the address/location data from lat and lon; not concerned with address being exactly corrent, just need correct county
        location_info = geolocator.reverse((r['Latitude'],r['Longitude']))
        location_info = location_info.address.split(", ")
        # some irreconcilable differences between geopy naming and US fips data (imported above) name; so have to semi-manually/manually assign these county names
        if r['Facility State or Province'] == 'LA':
            county_name = [s for s in location_info if 'Parish' in s][0].replace(' Parish','')
        elif (r['Facility State or Province'] == 'MO') and (r['Facility City'] == 'St. Louis'):
            county_name = 'St Louis City'
        elif (r['Facility State or Province'] == 'CO') and (r['Facility City'] == 'Denver'):
            county_name = 'Denver'
        elif (r['Facility State or Province'] == 'NV') and (r['Facility City'] == 'Carson City'):
            county_name = 'Carson City'
        elif (r['Facility State or Province'] == 'NV') and (r['Facility City'] == 'Clayton Valley'):
            county_name = 'Esmeralda'
        elif (r['Facility State or Province'] == 'NV') and ('Silver Peak' in r['Facility City']):
            county_name = 'Esmeralda'
        elif (r['Facility State or Province'] == 'CA') and ((r['Facility City'] == 'Los Angeles') or (r['Facility City'] == 'Valencia') or (r['Facility City'] == 'Santa Fe Springs')):
            county_name = 'Los Angeles'
        else:
            county_name = [s for s in location_info if 'County' in s][0].replace(' County','')
        county_name = re.sub(r'[^\w\s]','',county_name)

        if county_name == 'DuPage':
            county_name = 'Du Page'    
        elif county_name == 'Saint Clair':
            county_name = 'St Clair'

        # get state fip using dictionary above and state abbr in NAATBatt
        state_fips = state_fips_codes[r['Facility State or Province']]
        df.loc[i,'State_FIPS'] = state_fips
        # get county fip from subsetting US fips data using the state fip and county name
        county_fips_info = df_fips[(df_fips['FIPS State'] == state_fips) & (df_fips['County Name'] == county_name)].reset_index(drop=True)
        county_fips = county_fips_info.loc[0,'FIPS County']
        df.loc[i,'County_FIPS'] = county_fips
        # get state and county fip combined
        df.loc[i,'State_County_FIPS'] = state_fips + county_fips
        
        # append data to list
        state_county_FIP.append(state_fips+county_fips)

    # append data to list
    dfs.append(df)



# get all unique state county FIP numbers incase there is any repetition in the data, we only call census api once
state_county_FIPs = set(state_county_FIP)
state_county_FIPs = list(state_county_FIPs)

# save cleaned naatbatt data, locations of actual facility and locations used to get county/blockgroup data
with pd.ExcelWriter('Data/naatbatt_clean_locations/naatbatt_cleaned_with_locations.xlsx') as writer:
    dfs[0].to_excel(writer,sheet_name='1-RawMatl')
    dfs[1].to_excel(writer,sheet_name='2-BGMatl')
    dfs[2].to_excel(writer,sheet_name='3-BComp')
    dfs[3].to_excel(writer,sheet_name='4-ElectrodeCell')
    dfs[4].to_excel(writer,sheet_name='5-ModPack')
    dfs[5].to_excel(writer,sheet_name='6-EOL')


## Get census data

In [ ]:
## variables and dictionaries

# census ACS api key
api_key = 'b363a4f73dc726801a4ad8d56b1cbf40e6ae9dc7'

# acs codes and meaning for demographic/characteristic data; for any level of geography
acs_codes = {'B01003_001E':'totalPop',
            'B02001_002E':'whitePop',
            'B02001_003E':'blackPop',
            'B02001_004E':'indigenousPop',
            'B02001_005E':'asianPop',
            'B02001_006E':'hawaiiPacificPop',
            'B03001_003E':'hispanicPop',
            #'B03001_027E':'spaniardPop',

            'B01001_002E':'malePop',
            'B01001_026E':'femalePop',

            'B06009_002E':'lessHighSchl',
            'B06009_003E':'highSchl',
            'B06009_004E':'someUni',
            'B06009_005E':'uniBach',
            'B06009_006E':'uniGrad',

            'B17001_002E':'incBelowPov',
            'C17002_002E':'incLessHalfPov',
            'C18120_006E':'unemployed',

            'B27001_004E':'menHealthCare1',
            'B27001_007E':'menHealthCare2',
            'B27001_010E':'menHealthCare3',
            'B27001_013E':'menHealthCare4',
            'B27001_016E':'menHealthCare5',
            'B27001_019E':'menHealthCare6',
            'B27001_022E':'menHealthCare7',
            'B27001_025E':'menHealthCare8',
            'B27001_028E':'menHealthCare9',
            'B27001_032E':'womenHealthCare1',
            'B27001_035E':'womenHealthCare2',
            'B27001_038E':'womenHealthCare3',
            'B27001_041E':'womenHealthCare4',
            'B27001_044E':'womenHealthCare5',
            'B27001_047E':'womenHealthCare6',
            'B27001_050E':'womenHealthCare7',
            'B27001_053E':'womenHealthCare8',
            'B27001_056E':'womenHealthCare9',

            'C18108_003E':'disabled1',
            'C18108_007E':'disabled2',
            'C18108_011E':'disabled3',

            'B24031_004E':'miningOGJob',

            'tract]':'tract',
            'block group]':'blockGroup',
            '[["NAME"':'Name'
}

# to save the blockgroup and tract dfs by their state county pairings
sc_fip_tract_data_dict = dict()
sc_fip_census_data_dict = dict()



### Census data from the American Community Survey 2022 at the Block Group level
getting census data at the block group level using state and county fips codes for facilities in NAATBatt


In [ ]:
## block group and tract api calls and df organization

# block group data
for x in state_county_FIPs:
    # census ACS api, 5 year avg 2017-2022, at blockgroup level
    df = pd.read_csv(r"https://api.census.gov/data/2022/acs/acs5?get=NAME,B01003_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B03001_003E,B01001_002E,B01001_026E,B06009_002E,B06009_003E,B06009_004E,B06009_005E,B06009_006E,B17001_002E,C17002_002E,C18120_006E,B27001_004E,B27001_007E,B27001_010E,B27001_013E,B27001_016E,B27001_019E,B27001_022E,B27001_025E,B27001_028E,B27001_032E,B27001_035E,B27001_038E,B27001_041E,B27001_044E,B27001_047E,B27001_050E,B27001_053E,B27001_056E,C18108_003E,C18108_007E,C18108_011E,B24031_004E&for=block%20group:*&in=county:" + x[2:] + '&in=state:' + x[0:2] + '&key=' + api_key)
    df.rename(columns = acs_codes,inplace=True)

    # arithmetic transformations, get single estimate for characteristics
    df['menHealthCare'] = df['menHealthCare1'] + df['menHealthCare2'] + df['menHealthCare3'] + df['menHealthCare4'] + df['menHealthCare5'] + df['menHealthCare6'] + df['menHealthCare7'] + df['menHealthCare8'] + df['menHealthCare9']
    df['womenHealthCare'] = df['womenHealthCare1'] + df['womenHealthCare2'] + df['womenHealthCare3'] + df['womenHealthCare4'] + df['womenHealthCare5'] + df['womenHealthCare6'] + df['womenHealthCare7'] + df['womenHealthCare8'] + df['womenHealthCare9']
    df['disabledPop'] = df['disabled1'] + df['disabled2'] + df['disabled3']
    # drop uneccessary columns
    df.drop(columns = ['womenHealthCare1','womenHealthCare2','womenHealthCare3','womenHealthCare4','womenHealthCare5','womenHealthCare6','womenHealthCare7','womenHealthCare8','womenHealthCare9','menHealthCare1','menHealthCare2','menHealthCare3','menHealthCare4','menHealthCare5','menHealthCare6','menHealthCare7','menHealthCare8','menHealthCare9','disabled1','disabled2','disabled3'],inplace=True)

    df['state'] = df['state'].apply(lambda x: str(x))
    df['county'] = df['county'].apply(lambda x: str(x))
    df['tract'] = df['tract'].apply(lambda x: str(x))
    df['blockGroup'] = df["blockGroup"].str.replace("]", "")
    df['Name'] = df['Name'].str.replace('["','')

    df['state'] = df['state'].str.zfill(2)
    df['county'] = df['county'].str.zfill(3)
    df['tract'] = df['tract'].str.zfill(6)
    df['blockGroup'] = df['blockGroup'].str.zfill(5)
    df["GEOID"] = df["state"] + df["county"] + df['tract'] + df["blockGroup"]

    # get individual df per block group
    sc_fip_census_data_dict.update({x:df})


# save block group data
for x in sc_fip_census_data_dict:
    df = sc_fip_census_data_dict[x]
    df = df[df.columns.drop(list(df.filter(regex='Unnamed')))]
    df.to_csv('Data/census_by_state_county_FIP/block_group_from_' + x + '.csv')

## Merge census data and NAATBatt data
get geospatial census data then merge with block group geometries with nattbatt data

In [ ]:
# see what states have facilities anywhere along supply chain
states_facilities = list(set([x[0:2] for x in list(sc_fip_census_data_dict.keys())]))

df_geos = []
df_geos_dict = dict() # dict to save geometries

# go through all the states with facilities to get their ACS 2022 tiger line geometries
for x in states_facilities:
    df = gpd.read_file('Data/census_geometries/bg/tl_2023_'+str(x)+'_bg/tl_2023_'+str(x)+'_bg.shp')
    df_geos.append(df)
    df_geos_dict.update({x:df})


dfs_nb_geo = []

# make naatbatt data geo df using facility lat and lon coordinates
for x in dfs:
    gdf = gpd.GeoDataFrame(x, geometry=gpd.points_from_xy(x['Facility Lon'], x['Facility Lat']), crs="EPSG:4326")
    gdf = gdf[gdf.columns.drop(list(gdf.filter(regex='Unnamed')))]
    dfs_nb_geo.append(gdf)


# spatially join the naatbatt data with ACS 2022 geometries, block group level
dfs_nb_census = []
for x in dfs_nb_geo:
    polygons_gdf = []
    for i,r in x.iterrows():
        state = r['State_FIPS']
        state_geo = df_geos_dict[state]
        state_geo = state_geo.to_crs(4326)
        row_gdf = x[x['ID'] == r['ID']]
        row_update = gpd.sjoin(row_gdf,state_geo,how='right',predicate='within')
        row_update = row_update[~(row_update['ID'].isna())]
        polygons_gdf.append(row_update)
    gdf_points_polygons = gpd.GeoDataFrame(pd.concat(polygons_gdf,ignore_index=True),geometry='geometry',crs="EPSG:4326")
    gdf_points_polygons.drop(columns=['index_left','Supply Chain Segment', 'Company','NAATBatt Member','Facility Country','HQ Company', 'Sources', 'Notes', 'County'],inplace=True)
    dfs_nb_census.append(gdf_points_polygons)


# save data
dfs_nb_census[0].to_file('Data/naatbatt_census_shpfiles_bg/naatbatt_census_rawmatl.shp', driver='ESRI Shapefile')
dfs_nb_census[1].to_file('Data/naatbatt_census_shpfiles_bg/naatbatt_census_bmatl.shp', driver='ESRI Shapefile')
dfs_nb_census[2].to_file('Data/naatbatt_census_shpfiles_bg/naatbatt_census_bcomp.shp', driver='ESRI Shapefile')
dfs_nb_census[3].to_file('Data/naatbatt_census_shpfiles_bg/naatbatt_census_electrodcell.shp', driver='ESRI Shapefile')
dfs_nb_census[4].to_file('Data/naatbatt_census_shpfiles_bg/naatbatt_census_modpack.shp', driver='ESRI Shapefile')
dfs_nb_census[5].to_file('Data/naatbatt_census_shpfiles_bg/naatbatt_census_eol.shp', driver='ESRI Shapefile')


# merge block level census data to each row of dataframes
dfs_nb_census_bg = []
for x in dfs_nb_census:
    df_formerge = pd.DataFrame()
    for i,r in x.iterrows():
        sc_code = r['State_FIPS'] + r['County_FIPS']
        df = pd.read_csv('Data/census_by_state_county_FIP/block_group_from_'+sc_code+'.csv',converters={'state': str,'county':str,'tract':str})
        df['blockGroup'] = df['blockGroup'].apply(str)
        df['geoid_formerge'] = df['state'] + df['county'] + df['tract'] + df['blockGroup']
        bg_code = r['GEOID']
        df = df[df['geoid_formerge'] == bg_code]
        df_formerge = pd.concat([df_formerge,df])
    dfint = x.merge(df_formerge,left_on='GEOID',right_on='geoid_formerge',how='left')
    dfint.drop_duplicates(inplace=True)
    dfint.drop(columns=['GEOID_y','geoid_formerge'])
    dfint = gpd.GeoDataFrame(dfint)
    dfs_nb_census_bg.append(dfint)

dfs_nb_census_bg[0].to_file('Data/naatbatt_census_demographic_shpfiles_bg/naatbatt_census_rawmatl.shp', driver='ESRI Shapefile')
dfs_nb_census_bg[1].to_file('Data/naatbatt_census_demographic_shpfiles_bg/naatbatt_census_bmatl.shp', driver='ESRI Shapefile')
dfs_nb_census_bg[2].to_file('Data/naatbatt_census_demographic_shpfiles_bg/naatbatt_census_bcomp.shp', driver='ESRI Shapefile')
dfs_nb_census_bg[3].to_file('Data/naatbatt_census_demographic_shpfiles_bg/naatbatt_census_electrodcell.shp', driver='ESRI Shapefile')
dfs_nb_census_bg[4].to_file('Data/naatbatt_census_demographic_shpfiles_bg/naatbatt_census_modpack.shp', driver='ESRI Shapefile')
dfs_nb_census_bg[5].to_file('Data/naatbatt_census_demographic_shpfiles_bg/naatbatt_census_eol.shp', driver='ESRI Shapefile')


## Analysis

### compare facility demographic data with US statistics

In [ ]:
us_total = 333287562 # ACS 5-year 2022 US total population

# poverty status calculated using population of 325521470
us_pct_poverty = 40951625/325521470 # 40951625 people in US below poverty line
us_pct_poorest = 19950754/325521470 # 19950754 people in US below 50% of poverty line

us_white = 202889017/us_total
us_black = 40603656/us_total
us_asian = 19696980/us_total
us_hawaiian = 665807/us_total
us_indigenous = 3205331/us_total
us_hispanic = 63553639/us_total

us_women = 168059348/us_total
us_men = 165228214/us_total

us_lesshigh = (3700038+10742781+13856917)/us_total
us_high = (10553243+59741825)/us_total
us_somecollege = (13110313+44692390)/us_total
us_bachelor_more = (3919302+47391673+30359674)/us_total

us_unemp = 8944003/us_total

us_disabled = 41941456/us_total

us_healthinswomen = 165482854/us_total
us_healthinsmen = 160664656/us_total


segment_dict = {0:'Raw Material',1:'Material Processing',2:'Battery Components',3:'Electrodes and Cells',4:'Pack, Module, and Systems',5:'End of Life'}


##############################
### Graphing
##############################

dfs_nb_demograph_bg_figures = []
k=0
for x in dfs_nb_census_bg:
    dfint = x.copy()
    dfint = dfint[['ID', 'Status', 'Facility Name', 'Product','Facility Lat', 'Facility Lon','STATEFP','COUNTYFP',
        'TRACTCE', 'BLKGRPCE', 'GEOID_x', 'GEOIDFQ','geometry','Name', 'totalPop', 'whitePop', 'blackPop',
        'indigenousPop', 'asianPop', 'hawaiiPacificPop', 'disabledPop', 'malePop', 'femalePop', 'incLessHalfPov']]
    dfint['segment'] = segment_dict[k]
    dfint['blackPerc'] = dfint['blackPop']/dfint['totalPop']
    dfint['asianPerc'] = dfint['asianPop']/dfint['totalPop']
    dfint['whitePerc'] = dfint['whitePop']/dfint['totalPop']
    dfint['hawaiianPerc'] = dfint['hawaiiPacificPop']/dfint['totalPop']
    dfint['indigenousPerc'] = dfint['indigenousPop']/dfint['totalPop']
    dfint['poorestPerc'] = dfint['incLessHalfPov']/dfint['totalPop']

    dfs_nb_demograph_bg_figures.append(dfint)
    k=k+1


fig = plt.figure()
dfs_nb_demograph_bg_figures[0].plot(column="povPerc",missing_kwds = dict(color = "lightgrey",))
# fig = plt.figure()
# dfs_nb_demograph_tract_figures[1].plot(column="povPerc",missing_kwds = dict(color = "lightgrey",))
# fig = plt.figure()
# dfs_nb_demograph_tract_figures[2].plot(column="povPerc",missing_kwds = dict(color = "lightgrey",))
# fig = plt.figure()
# dfs_nb_demograph_tract_figures[3].plot(column="povPerc",missing_kwds = dict(color = "lightgrey",))
# fig = plt.figure()
# dfs_nb_demograph_tract_figures[4].plot(column="povPerc",missing_kwds = dict(color = "lightgrey",))
# fig = plt.figure()
# dfs_nb_demograph_tract_figures[5].plot(column="povPerc",missing_kwds = dict(color = "lightgrey",))


### Statistical testing

In [ ]:

state_facilities = []
for x in dfs_nb_census_bg:
    state_facilities.append(dict(x['state'].value_counts()))

df_states = pd.DataFrame(columns=state_fips_codes.keys())
df_states = df_states.transpose()
df_states.reset_index(inplace=True)
df_states.rename(columns={'index':'states'},inplace=True)
df_states['fips'] = df_states['states'].map(state_fips_codes)
df_states['Extraction'] = df_states['fips'].map(state_facilities[0])
df_states['Processing'] = df_states['fips'].map(state_facilities[1])
df_states['CellComponents'] = df_states['fips'].map(state_facilities[2])
df_states['ElectrodeCell'] = df_states['fips'].map(state_facilities[3])
df_states['ModulePackSystem'] = df_states['fips'].map(state_facilities[4])
df_states['EOL'] = df_states['fips'].map(state_facilities[5])
df_states.replace(np.nan,0,inplace=True)


# average block group demographic percentages per life cycle stage
demographic_avgs = []
for x in dfs_nb_census_bg:
    x['whitePerc'] = x['whitePop']/x['totalPop']
    x['blackPerc'] = x['blackPop']/x['totalPop']
    x['indigenousPerc'] = x['indigenousPop']/x['totalPop']
    x['asainPerc'] = x['asianPop']/x['totalPop']
    x['hawaiiPacificPerc'] = x['hawaiiPacificPop']/x['totalPop']
    x['malePerc'] = x['malePop']/x['totalPop']
    x['femalePerc'] = x['femalePop']/x['totalPop']
    x['incLessHalfPovPerc'] = x['incLessHalfPov']/x['totalPop']
    x = x[['whitePerc', 'blackPerc', 'indigenousPerc',
       'asainPerc', 'hawaiiPacificPerc', 'malePerc', 'femalePerc',
       'incLessHalfPovPerc']]
    df_avgs = pd.DataFrame(x.mean())
    df_avgs[0] = df_avgs[0]*100
    demographic_avgs.append(df_avgs)

dem_avgs = pd.DataFrame()
dem_avgs['Extraction'] = demographic_avgs[0][0].to_list()
dem_avgs['Processing'] = demographic_avgs[1][0].to_list()
dem_avgs['CellComponents'] = demographic_avgs[2][0].to_list()
dem_avgs['ElectrodesCells'] = demographic_avgs[3][0].to_list()
dem_avgs['ModulePackSystem'] = demographic_avgs[4][0].to_list()
dem_avgs['EOL'] = demographic_avgs[5][0].to_list()
dem_avgs.set_index(df_avgs[0].index,inplace=True)

dem_avgs_1 = dem_avgs.transpose()


##############################
### Graphing
##############################


fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(111, frameon=True, xticks=[], yticks=[])

the_table=plt.table(cellText=vals, rowLabels=df.index, colLabels=df.columns, 
                    colWidths = [0.03]*vals.shape[1], loc='center', 
                    cellColours=colours)
plt.show()